In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from size_matters.aggregation.aggregators import apply_condorcet_aggregator
from size_matters.evaluation.accuracy import compare_methods
import pandas as pd
from pathlib import Path
from size_matters.utils.inventory import COLUMNS

In [ ]:
FOLDER_PATH = Path("../data/countries")
ALTERNATIVES = ["France", "Tunisia", "Egypt", "Greece", "Spain"]


In [ ]:
raw = pd.read_csv(FOLDER_PATH / "raw.csv")
raw = raw.drop(columns=["Horodateur","Score"])

In [ ]:
groundtruth = raw[raw["Choose you pseudo"]=="GroudTruth"].drop(columns=["Choose you pseudo"])
annotations = raw[raw["Choose you pseudo"]!="GroudTruth"].drop(columns=["Choose you pseudo"])

### Prepare ground truth

In [ ]:
groundtruth = groundtruth.T
groundtruth[0] = pd.Categorical(groundtruth[0])
groundtruth = pd.get_dummies(groundtruth[0])
groundtruth.index.name = COLUMNS.question

In [ ]:
groundtruth.to_csv(FOLDER_PATH / "groundtruth.csv")

### Prepare Annotations

In [ ]:
annotations = annotations.T
annotations = annotations.stack()
annotations.index.names = [COLUMNS.question, COLUMNS.voter]

multi_answers  = annotations.str.split(", ",expand=False)
annotations = pd.DataFrame(index=annotations.index, columns=ALTERNATIVES)
annotations = annotations.apply(lambda x: multi_answers.apply(lambda y: int(x.name in y)))

In [ ]:
annotations.to_csv(FOLDER_PATH / "annotations.csv")